In [1]:
import os
import sys

# certify the notebook is able to find "deck_class" folder
current_directory = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_directory, '..')))

In [2]:
from deck_class.truco import Deck_of_Truco
from dataclasses import dataclass
import numpy as np

# Game modeling

Similar to `struct` in C, Python has the native `dataclasses` module that implements the `@dataclass` decorator for automatically adding generated special methods such as `__init__()`.

**Player** attributes:
1. `hand` -> list with three cards
2. `index` -> int value for reference (0 or 1)


**Team** attributes:
1. `players` -> list with two instances of **Player**
2. `index` -> int value for reference (0 or 1)

For ingame reference, each player will be indentified by the tuple `(Team.index, Player.index)`.

In [20]:
@dataclass
class Player:
    hand: list
    index: int
    
    
@dataclass
class Team:
    players: list
    index: int

# Class `Truco_Game`

This class will consist of handling the game state, certifying that the rules of the game are followed and providing methods that enable the agents to play the game.

* Definition of state ($s$):
1. $P_s$ = player that can take action at the current state $s$
2. $C_P$ = player and teammate's cards 
3. $p_0$ = points earned by team 0
4. $p_1$ = points earned by team 1
5. $p_s$ = points at stake in the current round
6. $T_C$ = current cards on the table
7. $F_P$ = first player of current round (mão)
8. $B$ = if the previous player raised the round value
9. $W$ = array with the team index which won $i$-th round ($W = [w_0, w_1, w_2]$)
10. $n_r$ = number of round current beeing played

$$s = (P_s, C_P, p_0, p_1, p_s, T_C, F_P, B, W, n_r)$$

Obs.: if the class was instantiated with 4 players $C_P$ will hide teammates cards. 

In [ ]:
def mode(array):
    counter = {}
    
    for element in array:
        if element is None or element == -1:
            continue
        
        if element in counter.keys():
            counter[element] += 1
        else:
            counter[element] = 1
    
    counter = dict(sorted(counter.items(), key=lambda item: item[1], reverse=True))
    mode_value, mode_count = list(counter.items())[0]
    return mode_value

In [71]:
class Truco_Game():
    
    def __init__(self, max_points=12, n_players=2):
        self._deck = Deck_of_Truco()        
        self._goal = max_points
        self._teams = self.draw_teams_hands()
        self._first_to_play = (0, 0)
        self._last_to_play = (1, 1)
        self._screamer = None
        self._n_players = n_players
        
        
    def draw_teams_hands(self):
        player_00 = Player(self._deck.draw_hand(), 0)
        player_01 = Player(self._deck.draw_hand(), 0)
        player_10 = Player(self._deck.draw_hand(), 1)
        player_11 = Player(self._deck.draw_hand(), 1)

        
#         player_00 = Player(np.array(['H3', 'DA', 'C4']), 0)
#         player_01 = Player(np.array(['D7', 'D2', 'SQ']), 0)
#         player_10 = Player(np.array(['CJ', 'HJ', 'DK']), 1)
#         player_11 = Player(np.array(['DQ', 'SA', 'C3']), 1)
        
        team_0 = Team([player_00, player_01], 0)
        team_1 = Team([player_10, player_11], 1)
        
        return [team_0, team_1]
    
        
    def start_state(self):
        first_player = self._first_to_play
        player_hand = self.get_team_hand(first_player)
        
        points_t1, points_t2 = 0, 0
        round_value = 1
        cards_on_table = []
        raise_call = 0
        
        w_1 = None
        w_2 = None
        w_3 = None
        
        n_round = 0
        
        return (first_player, player_hand, points_t1, points_t2, round_value,
                cards_on_table, (0, 0), raise_call, [w_1, w_2, w_3], n_round)
    
    
    def get_team_hand(self, player):
        team_index = player[0]
        player_index = player[1]
        teammate_index = (player_index + 1) % 2
        
        hand_1 = self._teams[team_index].players[player_index].hand
        
        if self._n_players == 2:
            hand_2 = self._teams[team_index].players[teammate_index].hand
        else:
            hand_2 = ['*', '*', '*']
        
        
        return (hand_1, hand_2)
    
    
    def successor(self, state, action):
        player, team_hand, points_t0, points_t1, round_value,\
        table_cards, first_player, raise_call, round_winners,\
        n_round = state
        
        current_team, current_player = player
        player_hand = team_hand[0]
        
        # game logic regarding the action 'playing card'
        if action in player_hand:
            card_played = action
            
            self.remove_card_from_hand(player, card_played)
            next_player = self.forward_player(player)
            table_cards.append(card_played)
            
            if len(table_cards) == 4:
                winner_index = self.index_winner_card(table_cards)
                
                if len(winner_index) == 1:
                    winner_index = winner_index[0]
                    
                    winner_player = self.find_player(winner_index, first_player)
                    round_winner_team = winner_player[0]
                    round_winners[n_round] = round_winner_team
                    
                    cond_all_round_winners = (round_winners.count(None) == 0)
                    cond_b2b_wins = (round_winners[0] == round_winners[1])
                    cond_canga_win = (round_winners[0] == -1 and round_winners[1] is not None)
                    
                    if cond_all_round_winners or cond_b2b_wins or cond_canga_win:
                        hand_winner_team = mode(round_winners)

                        if hand_winner_team == 1:
                            points_t1 += round_value
                        else:
                            points_t0 += round_value

                        raise_call, round_value = 0, 1
                        round_winners = [None, None, None]

                        self.start_new_hand()
                        next_player = self._first_to_play 
                        n_round = 0
                    else:
                        next_player = winner_player
                        first_player = winner_player
                        
                        n_round += 1

                    table_cards = []
                
                # game 'cangado'
                else:
                    last_canga_index = winner_index[-1]
                    canga_player = self.find_player(last_canga_index, first_player)
                    
                    if n_round >= 1:
                        hand_winner_team = round_winners[0]
                        
                        if hand_winner_team == 1:
                            points_t1 += round_value
                        else:
                            points_t0 += round_value

                        raise_call, round_value = 0, 1
                        round_winners = [None, None, None]

                        self.start_new_hand()
                        next_player = self._first_to_play
                        table_cards = []
                        n_round = 0
                    
                    else:
                        next_player = canga_player
                        first_player = canga_player
                        
                        round_winners[0] = -1
                        n_round += 1
                        table_cards = []
                
        # game logic regarding the action 'run'
        elif action == 'run':
            if current_team == 0:
                points_t1 += round_value
            else:
                points_t0 += round_value
                
            raise_call, round_value = 0, 1
            table_cards = []
            round_winners = [None, None, None]
            
            self.start_new_hand()
            next_player = self._first_to_play
            n_round = 0
        
        # game logic regarding the action 'raise'
        elif action == 'raise':
            # TODO: criar novo estado em que time decide se vai ou nao
            pass
        
        # game logic regarding the action 'go'
        elif action == 'go':
            raise_call = 0
            
            next_player = self._screamer
            self._screamer = None
            
            if round_value == 1:
                round_value = 3
            else:
                round_value += 3
       
    
        next_hand = self.get_team_hand(next_player)
        
        return (next_player, next_hand, points_t0, points_t1,\
                round_value, table_cards, first_player, raise_call,\
                round_winners, n_round)
    
    
    def start_new_hand(self):
        self._deck.reset_cards()
        self._teams = self.draw_teams_hands()
        
        self._first_to_play = self.forward_player(self._first_to_play)
        self._last_to_play = self.forward_player(self._last_to_play)
        return
    
    
    def remove_card_from_hand(self, player, card):
        hand = self._teams[player[0]].players[player[1]].hand
        self._teams[player[0]].players[player[1]].hand = np.delete(hand, np.where(hand == card))
        return
             
    
    def forward_player(self, player):
        team_index = player[0]
        player_index = player[1]
        
        if team_index == 1:
            new_player_index = (player_index + 1)%2
        else:
            new_player_index = player_index
        
        new_team_index = (team_index + 1)%2
        
        return (new_team_index, new_player_index)
    
    
    def possible_actions(self, state):
        player, team_hand, points_t0, points_t1, round_value,\
        table_cards, first_player, raise_call, round_winners,\
        n_round = state
        
        player_hand = team_hand[0]
        
        if raise_call:
            possible_actions = ['go', 'run']
        else:
            possible_actions = list(player_hand)
            
        if ((round_value < 9) or (round_value == 9 and not raise_call)) and max(points_t0, points_t1) < 11:
            possible_actions.append('raise')
        
        return possible_actions
        
    
    
    def index_winner_card(self, cards_played):
        card_forces = get_card_forces(cards_played)

        strongest_card_index = np.argwhere(card_forces == np.amax(card_forces))\
                                 .flatten()\
                                 .tolist()

        winner_index = strongest_card_index
        
        if len(strongest_card_index) == 2:
            # check if teammates had same card
            if (strongest_card_index[0] + 2) == strongest_card_index[1]:
                winner_index = [strongest_card_index[1]]

        return winner_index
    
    
    def get_card_forces(self, cards):
        card_force = {
            'C4': 10,
            'H7': 9,
            'SA': 8,
            'D7': 7,
            '3': 6,
            '2': 5,
            'A': 4,
            'K': 3,
            'Q': 2,
            'J': 1
        }

        values = [
            card_force[card]
            if card in card_force.keys() else card_force[card[1]]
            for card in cards
        ]

        return values
    
    
    def find_player(self, index, first_player): 
        player = first_player
        
        while index > 0:
            player = self.forward_player(player)
            index -= 1
            
        return player
    
      
    def is_end(self, state):
        points_t1, points_t2 = state[2], state[3]
        return points_t1 >= 12 or points_t2 >= 12
       

In [72]:
def format_action(action_string):
    if len(action_string) == 2:
        return action_string.upper()
    
    return action_string.lower()


def humanPolicy(game, state):
    while True:
        print('\n\n## POINTS ##')
        print(f'Team 0 - {state[2]}\nTeam 1 - {state[3]}\n\n')
        
        print('Current State:')
        print(f'Player -> {state[0]}')
        print(f'Points at stake -> {state[4]}')
        print(f'Player\'s cards -> {state[1]}')
        
        print(f'\nCards in table -> {state[5]}')
        print(f'Round winners -> R1: {state[8][0]} | R2: {state[8][1]} | R3: {state[8][2]}')
        
        possible_actions = game.possible_actions(state)
        action = input(f'Possible actions -> {possible_actions}: ')
        action = format_action(action)
        
        if action in possible_actions:
            return action
        else:
            clear_output(wait = True)
            print('Invalid action! Choose again!')
            

def run_game():
    game = Truco_Game()
    state = game.start_state()
    
    while not game.is_end(state):
        action = humanPolicy(game, state)
        state = game.successor(state, action)
        clear_output(wait = True)

In [73]:
run_game()